In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 데이터분석
import pandas as pd
import numpy as np 
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# 모델들
import lightgbm as lgb
import xgboost
!pip install catboost
import catboost 
# 모델 성능 측정을위한
from sklearn.metrics import mean_absolute_error 
# feature 중요도 시각화
from lightgbm import plot_importance
# 램용량 확보
import gc
# 하이퍼파라미터 세팅(optuna)
!pip install optuna
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
# train test split
from sklearn.model_selection import train_test_split
# 앙상블모델
from sklearn.ensemble import RandomForestRegressor
# 스태킹(KFold)
from sklearn.model_selection import KFold
# 사이킷런에서 제공하는 점수제공 매써드 확인
import sklearn.metrics
# sorted(sklearn.metrics.SCORERS.keys())
# 차원축소
from sklearn.decomposition import PCA
# 정규화 진행
from sklearn.preprocessing import MinMaxScaler
# 교차검증
from sklearn.model_selection import cross_val_score
# 표준화
from sklearn.preprocessing import StandardScaler

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
# 결측치 제거(1개)
train=train.drop(index=train[train['winPlacePerc'].isnull()].index)

In [ ]:
# 걷지않았는데 kill이 있을경우
train=train.drop(index=train[ (train['walkDistance']==0) & (train['kills'] > 0)  ].index)
# 매치 최대 킬수가 해당경기 인원보다 많은경우
train=train.drop(index=train[ train.groupby('matchId')['kills'].transform('max')  > train.groupby('matchId')['Id'].transform('count')  ].index)
# rideDistance가 0인데 로드킬이 있는경우
train=train.drop(index=train[ (train['rideDistance']==0) & (train['roadKills']>0)  ].index)
# 데미지를 주지않았는데 킬이 있는경우
train=train.drop(index=train[ (train['damageDealt']==0) & (train['kills']>0)  ].index)

In [ ]:
# 그룹 사이즈를 표시하는 컬럼을 만듭니다
train['group_size']=train.groupby('groupId')['Id'].transform('count')
test['group_size']=test.groupby('groupId')['Id'].transform('count')

# 게임내 오류로 만들어진 이상치를 나타내는 컬럼을 만듭니다

# 조건1: numGroups와 maxPlace가 다를경우
cond1=train['numGroups']!=train['maxPlace']
cond11=test['numGroups']!=test['maxPlace']
# 조건2: 한그룹에 5명이상 존재할경우
cond2=train['group_size']>4
cond22=test['group_size']>4

# outlier
train.loc[ cond1 , 'outlier']=1
train['outlier'] = train['outlier'].fillna(0)
test.loc[ cond11 , 'outlier']=1
test['outlier'] = test['outlier'].fillna(0)

# outlier1
train.loc[ cond2 , 'outlier1']=1
train['outlier1'] = train['outlier1'].fillna(0)
test.loc[ cond22 , 'outlier1']=1
test['outlier1'] = test['outlier1'].fillna(0)

In [ ]:
# matchDuration 맵의 크기에 따라 0과 1로 분류 
def func(x):
    if x < 1625:
        return 0
    else:
        return 1
train['matchDuration_class']=train['matchDuration'].map(func).to_frame()
test['matchDuration_class']=test['matchDuration'].map(func).to_frame()

In [ ]:
# 그룹별 각피쳐들의 평균
for i in['assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints',
       'kills', 'killStreaks', 'longestKill', 'matchDuration',  'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']:
    train[i+'_mean']=train.groupby('groupId')[i].transform('mean')

for i in['assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints',
       'kills', 'killStreaks', 'longestKill', 'matchDuration',  'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']:
    test[i+'_mean']=test.groupby('groupId')[i].transform('mean')

In [ ]:
# 평균화한 피쳐들의 매치안에서 랭킹으로 줄세우기(가장 값이 높을때 1등이 될수있도록 ascending을 조절했습니다)
for i in ['assists_mean', 'boosts_mean', 'damageDealt_mean',
        'DBNOs_mean', 'headshotKills_mean', 'heals_mean', 'killPlace_mean',
       'killPoints_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'matchDuration_mean', 
       'rankPoints_mean', 'revives_mean', 'rideDistance_mean',
       'roadKills_mean', 'swimDistance_mean', 'teamKills_mean',
       'vehicleDestroys_mean', 'walkDistance_mean',
       'weaponsAcquired_mean', 'winPoints_mean']:
    train[i+'_place']=train.groupby('matchId')[i].transform('rank', ascending=False)  

for i in ['assists_mean', 'boosts_mean', 'damageDealt_mean',
        'DBNOs_mean', 'headshotKills_mean', 'heals_mean', 'killPlace_mean',
       'killPoints_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'matchDuration_mean', 
       'rankPoints_mean', 'revives_mean', 'rideDistance_mean',
       'roadKills_mean', 'swimDistance_mean', 'teamKills_mean',
       'vehicleDestroys_mean', 'walkDistance_mean',
       'weaponsAcquired_mean', 'winPoints_mean']:
    test[i+'_place']=test.groupby('matchId')[i].transform('rank', ascending=False)

In [ ]:
# 그룹안에서 피쳐별 max값을 표시합니다 
for i in['assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints',
       'kills', 'killStreaks', 'longestKill', 'matchDuration', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']:
    train[i+'_max']=train.groupby('groupId')[i].transform('max')

for i in['assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints',
       'kills', 'killStreaks', 'longestKill', 'matchDuration',  'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']:
    test[i+'_max']=test.groupby('groupId')[i].transform('max')

In [ ]:
# 그룹안의 max값을 기준으로 매치안에서 랭킹으로 줄세우기를 합니다
for i in['assists_max', 'boosts_max',
       'damageDealt_max', 'DBNOs_max', 'headshotKills_max', 'heals_max',
       'killPlace_max', 'killPoints_max', 'kills_max', 'killStreaks_max',
       'longestKill_max', 'matchDuration_max', 
       'rankPoints_max', 'revives_max', 'rideDistance_max',
       'roadKills_max', 'swimDistance_max', 'teamKills_max',
       'vehicleDestroys_max', 'walkDistance_max', 'weaponsAcquired_max',
       'winPoints_max']:
    train[i+'_place']=train.groupby('matchId')[i].transform('rank', ascending=False)

for i in['assists_max', 'boosts_max',
       'damageDealt_max', 'DBNOs_max', 'headshotKills_max', 'heals_max',
       'killPlace_max', 'killPoints_max', 'kills_max', 'killStreaks_max',
       'longestKill_max', 'matchDuration_max', 
       'rankPoints_max', 'revives_max', 'rideDistance_max',
       'roadKills_max', 'swimDistance_max', 'teamKills_max',
       'vehicleDestroys_max', 'walkDistance_max', 'weaponsAcquired_max',
       'winPoints_max']:
    test[i+'_place']=test.groupby('matchId')[i].transform('rank', ascending=False)

In [ ]:
# 어시스트/매치인원: 10명있을때 2명 어시스트한것과 100명있을때 2명어시스트한것의 가치는 다르다
train['assists_per']= train['assists']  /  ( train.groupby('matchId')['Id'].transform('count') )
test['assists_per']= test['assists']  /  ( test.groupby('matchId')['Id'].transform('count') )
# 부스트 * 매치인원: 매치인원이 많을수록 부스트먹기가 힘들어집니다(경쟁자가 생김)
train['boosts_value']= train['boosts']  * ( train.groupby('matchId')['Id'].transform('count') )
test['boosts_value']= test['boosts']  * ( test.groupby('matchId')['Id'].transform('count') )
# 데미지양/매치에서발생한 총데미지: 매치안에서 나의 데미지가 차지하는정도
train['damageDealt_per']= train['damageDealt']  / ( train.groupby('matchId')['damageDealt'].transform('sum') )
train['damageDealt_per'] = train['damageDealt_per'].fillna(0)
test['damageDealt_per']= test['damageDealt']  / ( test.groupby('matchId')['damageDealt'].transform('sum') )
test['damageDealt_per'] = test['damageDealt_per'].fillna(0)
# DBNO/매치인원: DBNO의 가치
train['DBNOs_per']= train['DBNOs']  / ( train.groupby('matchId')['Id'].transform('count') )
test['DBNOs_per']= test['DBNOs']  / ( test.groupby('matchId')['Id'].transform('count') )
# 헤드샷킬/킬: 헤드샷 비율
train['headshot_rate'] = train['headshotKills'] / train['kills']
train['headshot_rate'] = train['headshot_rate'].fillna(0) # 킬이0인경우 0으로 나눌수없기에 결측치처리됨
test['headshot_rate'] = test['headshotKills'] / test['kills']
test['headshot_rate'] = test['headshot_rate'].fillna(0) # 킬이0인경우 0으로 나눌수없기에 결측치처리됨
# 힐 * 매치인원: 매치인원이 많아질수록 힐먹기가 힘들어짐(경쟁자생김)
train['heals_per']= train['heals']  * ( train.groupby('matchId')['Id'].transform('count') )
test['heals_per']= test['heals']  * ( test.groupby('matchId')['Id'].transform('count') )
# 킬플레이스 / 매치인원: 매치인원이 많아질수록 1등의 가치는 높아집니다
train['killPlace_per']= train['killPlace']  / ( train.groupby('matchId')['Id'].transform('count') )
test['killPlace_per']= test['killPlace']  / ( test.groupby('matchId')['Id'].transform('count') )
# 킬 / 매치인원
train['kills_per']= train['kills']  / ( train.groupby('matchId')['Id'].transform('count') )
test['kills_per']= test['kills']  / ( test.groupby('matchId')['Id'].transform('count') )
# 연속킬 / 매치인원
train['killStreaks_per']= train['killStreaks']  / ( train.groupby('matchId')['Id'].transform('count') )
test['killStreaks_per']= test['killStreaks']  / ( test.groupby('matchId')['Id'].transform('count') )
# 부활 / 그룹사이즈: 그룹원이적을수록 부활의 가치가 커짐
train['revives_per']= train['revives']  / ( train['group_size'] )
test['revives_per']= test['revives']  / ( test['group_size'] )
# 차탄거리 * 매치인원: 인원이 많아질수록 돌아다니기 힘들어짐
train['rideDistance_value']= train['rideDistance']  * ( train.groupby('matchId')['Id'].transform('count') )
test['rideDistance_value']= test['rideDistance']  * ( test.groupby('matchId')['Id'].transform('count') )
# 로드킬 / 매치인원
train['roadKills_per']= train['roadKills']  / ( train.groupby('matchId')['Id'].transform('count') )
test['roadKills_per']= test['roadKills']  / ( test.groupby('matchId')['Id'].transform('count') )
# 수영거리 * 매치인원
train['swimDistance_value']= train['swimDistance']  * ( train.groupby('matchId')['Id'].transform('count') )
test['swimDistance_value']= test['swimDistance']  * ( test.groupby('matchId')['Id'].transform('count') )
# 팀킬 / 그룹사이즈
train['teamKills_per']= train['teamKills']  / ( train['group_size'] )
test['teamKills_per']= test['teamKills']  / ( test['group_size'] )
# vehicleDestroys / 맵크기: 맵이 좁을수록 차가 드물게나옴=> 차부시기 힘듬
train['vehicleDestroys_per']= train['vehicleDestroys']  / ( train['matchDuration_class']+1 )
test['vehicleDestroys_per']= test['vehicleDestroys']  / ( test['matchDuration_class']+1 )
# 걸은 거리 * 매치인원수
train['walkDistance_value']= train['walkDistance']  * ( train.groupby('matchId')['Id'].transform('count') )
test['walkDistance_value']= test['walkDistance']  * ( test.groupby('matchId')['Id'].transform('count') )
# 무기 획득수 * 매치인원수
train['weaponsAcquired_per']= train['weaponsAcquired']  * ( train.groupby('matchId')['Id'].transform('count') )
test['weaponsAcquired_per']= test['weaponsAcquired']  * ( test.groupby('matchId')['Id'].transform('count') )

In [ ]:
# 조합 컬럼 평균화
for i in ['assists_per', 'boosts_value', 'damageDealt_per',
       'DBNOs_per', 'headshot_rate', 'heals_per', 'killPlace_per', 'kills_per',
       'killStreaks_per', 'revives_per', 'rideDistance_value', 'roadKills_per', 'swimDistance_value', 'teamKills_per', 'vehicleDestroys_per',
       'walkDistance_value', 'weaponsAcquired_per']:
    train[i+'_mean']=train.groupby('groupId')[i].transform('mean')

for i in ['assists_per', 'boosts_value', 'damageDealt_per',
       'DBNOs_per', 'headshot_rate', 'heals_per', 'killPlace_per', 'kills_per',
       'killStreaks_per', 'revives_per', 'rideDistance_value', 'roadKills_per', 'swimDistance_value', 'teamKills_per', 'vehicleDestroys_per',
       'walkDistance_value', 'weaponsAcquired_per']:
    test[i+'_mean']=test.groupby('groupId')[i].transform('mean')

In [ ]:
# 조합 컬럼 평균화후 랭크화
for i in['assists_per_mean',
       'boosts_value_mean', 'DBNOs_per_mean', 'heals_per_mean',
       'killPlace_per_mean', 'kills_per_mean', 'killStreaks_per_mean',
       'revives_per_mean', 'rideDistance_value_mean',
       'roadKills_per_mean', 'swimDistance_value_mean',
       'teamKills_per_mean', 'vehicleDestroys_per_mean',
       'walkDistance_value_mean', 'weaponsAcquired_per_mean','group_size']:
    train[i+'_place']=train.groupby('matchId')[i].transform('rank', ascending=False)

for i in['assists_per_mean',
       'boosts_value_mean', 'DBNOs_per_mean', 'heals_per_mean',
       'killPlace_per_mean', 'kills_per_mean', 'killStreaks_per_mean',
       'revives_per_mean', 'rideDistance_value_mean',
       'roadKills_per_mean', 'swimDistance_value_mean',
       'teamKills_per_mean', 'vehicleDestroys_per_mean',
       'walkDistance_value_mean', 'weaponsAcquired_per_mean','group_size']:
    test[i+'_place']=test.groupby('matchId')[i].transform('rank', ascending=False)

reduce_mem_usage(train)
reduce_mem_usage(test)

In [ ]:
def func(x):
    if x in ['squad-fpp','squad']:
        return 'squad'
    elif x in ['duo-fpp', 'duo']:
        return 'duo'
    elif x in ['solo-fpp', 'solo']:
        return 'solo'
    elif x in ['normal-squad-fpp', 'normal-squad', 'normal-duo-fpp', 'normal-duo', 'normal-solo-fpp', 'normal-solo']:
        return 'normal'
    elif x in ['crashfpp', 'crashtpp']:
        return 'crash'
    else:
        return 'flare'

train['matchType']=train['matchType'].map(func).to_frame()
test['matchType']=test['matchType'].map(func).to_frame()

In [ ]:
# 램 용량 확보
gc.collect()

In [ ]:
# 사본생성
train2=train.copy()
test1=test.copy()
# 칼럼 드랍
# train1=train1.drop(['assists', 'boosts', 'headshotKills','weaponsAcquired','headshotKills_mean','revives_mean','swimDistance_mean_place','assists_max','DBNOs_max','headshotKills_max','rankPoints_max','headshotKills_max_place','revives_max_place','roadKills_max_place','teamKills_max_place','headshot_rate','swimDistance_value','revives_per_mean_place','teamKills_per_mean_place','revives', 'killPoints_mean', 'swimDistance_max','winPoints_max','killPoints_max_place','rankPoints_max_place','roadKills_per','rideDistance_value_mean_place','assists_mean','headshotKills_mean_place','killPoints_max','weaponsAcquired_max','teamKills_per_mean','assists_per_mean_place'], axis=1)
# test1=test1.drop(['assists', 'boosts', 'headshotKills','weaponsAcquired','headshotKills_mean','revives_mean','swimDistance_mean_place','assists_max','DBNOs_max','headshotKills_max','rankPoints_max','headshotKills_max_place','revives_max_place','roadKills_max_place','teamKills_max_place','headshot_rate','swimDistance_value','revives_per_mean_place','teamKills_per_mean_place','revives', 'killPoints_mean', 'swimDistance_max','winPoints_max','killPoints_max_place','rankPoints_max_place','roadKills_per','rideDistance_value_mean_place','assists_mean','headshotKills_mean_place','killPoints_max','weaponsAcquired_max','teamKills_per_mean','assists_per_mean_place'], axis=1)

# 샘플링 
train1=train2.sample(n=500000)

# X_train1, Y_train1, X_test1 생성
train1=pd.get_dummies(train1, columns = ['matchType'])
test1=pd.get_dummies(test1, columns = ['matchType'])
a=train1.loc[:,['Id','groupId','matchId','maxPlace']]
b=test1.loc[:,['Id','groupId','matchId','maxPlace']]
train1=train1.drop(['groupId', 'Id', 'matchId'], axis=1)
test1=test1.drop(['groupId', 'Id', 'matchId'], axis=1)
X_train1 = train1.drop(['winPlacePerc'], axis=1) 
Y_train1 = train1['winPlacePerc'] 
X_test1  = test1
X_train1.shape, Y_train1.shape, X_test1.shape

In [ ]:
# 표준화 1
scaler = StandardScaler()
scaler.fit(X_train1)
X_train1 = pd.DataFrame(scaler.transform(X_train1))

In [ ]:
# 표준화 2
scaler = StandardScaler()
scaler.fit(X_test1)
X_test1 = pd.DataFrame(scaler.transform(X_test1))

In [ ]:
# validation과 train으로 나누기 차원축소 안하는경우
X_train, X_val, y_train, y_val = train_test_split(X_train1, Y_train1, test_size=0.2, random_state=42)
X_train.shape, X_val.shape

In [ ]:
catboost_model = catboost.CatBoostRegressor()
catboost_model.fit(X_train, y_train)
Y_pred = catboost_model.predict(X_test1)
train_pred = catboost_model.predict(X_train)
# 후처리하면 점수는 더 잘나오나?
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(y_train, train_pred)))

In [ ]:
# 케글용
sub1 = pd.DataFrame({
        "Id": b['Id'],
        'matchId': b['matchId'],
        'maxPlace': b['maxPlace'],
        'groupId': b['groupId'],
        "winPlacePerc": Y_pred
    })
sub1['winPlacePerc']=sub1.groupby('groupId')['winPlacePerc'].transform('mean')

submission = pd.DataFrame({
        "Id": b['Id'],
        "winPlacePerc": sub1['winPlacePerc']
    })
submission.to_csv('submission.csv', index=False)